<a href="https://colab.research.google.com/github/kenclements/praxis/blob/main/adaboost-twos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!nvidia-smi

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

import re

In [2]:
file = pd.read_csv('/Data/TWOS-dataset/email_ano/User1b.csv')

In [3]:
file

,label,body_anon,liwc_WC,liwc_Analytic,liwc_Clout,liwc_Authentic,liwc_Tone,liwc_WPS,liwc_Sixltr,liwc_Dic,...,liwc_Comma,liwc_Colon,liwc_SemiC,liwc_QMark,liwc_Exclam,liwc_Dash,liwc_Quote,liwc_Apostro,liwc_Parenth,liwc_OtherP
0,NORMAL,Hey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NORMAL,Hey guys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NORMAL,Yeah I think we should: 71J871LWL3) Backup UAH...,42.0,22.77,82.96,6.50,3.31,14.00,14.29,78.57,...,0.0,2.38,0.0,2.38,0.0,0.0,0.0,0.0,7.14,0.0
3,NORMAL,Okay so as protection we will just clear all i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NORMAL,Can M5P10WM4K7 the whole desktop with excel fi...,38.0,12.16,96.70,50.35,97.58,9.50,7.89,81.58,...,0.0,0.00,0.0,2.63,0.0,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,ANOMALY,I think it is a good idea if you guys can orga...,69.0,46.46,87.68,27.39,78.80,8.62,15.94,91.30,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
156,ANOMALY,Hey guys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,ANOMALY,Tue =WG4V688FUG V1AGEWA8CT Wed =WG4V688FUG OAP...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,ANOMALY,Hey friends,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
file.describe()

,liwc_WC,liwc_Analytic,liwc_Clout,liwc_Authentic,liwc_Tone,liwc_WPS,liwc_Sixltr,liwc_Dic,liwc_function,liwc_pronoun,...,liwc_Comma,liwc_Colon,liwc_SemiC,liwc_QMark,liwc_Exclam,liwc_Dash,liwc_Quote,liwc_Apostro,liwc_Parenth,liwc_OtherP
count,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,...,38.0,38.000000,38.0,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000
mean,38.236842,45.571316,57.531316,50.381579,49.145000,10.069737,11.750000,83.261579,52.178684,15.788947,...,0.0,0.190789,0.0,1.333947,1.992368,0.612632,0.125263,2.196842,0.769474,2.437368
std,14.302588,29.711298,31.055044,33.247178,38.184176,3.525474,6.342325,8.535449,8.330098,5.961802,...,0.0,0.663327,0.0,1.744670,5.190065,1.966312,0.772174,2.956800,2.968952,5.046803
min,21.000000,1.000000,1.000000,1.000000,1.000000,5.000000,0.000000,52.940000,32.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,26.250000,19.117500,29.270000,15.532500,25.770000,7.690000,7.917500,79.485000,48.202500,12.125000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,35.500000,47.960000,50.000000,52.245000,25.770000,8.855000,11.270000,85.245000,53.280000,16.245000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.820000,0.000000,0.000000
75%,46.000000,66.975000,85.852500,80.122500,87.857500,12.000000,15.340000,88.557500,57.500000,20.000000,...,0.0,0.000000,0.0,2.820000,2.710000,0.000000,0.000000,3.675000,0.000000,3.097500
max,80.000000,96.850000,99.000000,99.000000,99.000000,18.500000,26.470000,93.480000,67.390000,28.260000,...,0.0,2.700000,0.0,5.560000,30.430000,10.810000,4.760000,13.640000,16.220000,26.190000


In [5]:
file.index

RangeIndex(start=0, stop=160, step=1)

In [6]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 95 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   label              160 non-null    object 
 1   body_anon          160 non-null    object 
 2   liwc_WC            38 non-null     float64
 3   liwc_Analytic      38 non-null     float64
 4   liwc_Clout         38 non-null     float64
 5   liwc_Authentic     38 non-null     float64
 6   liwc_Tone          38 non-null     float64
 7   liwc_WPS           38 non-null     float64
 8   liwc_Sixltr        38 non-null     float64
 9   liwc_Dic           38 non-null     float64
 10  liwc_function      38 non-null     float64
 11  liwc_pronoun       38 non-null     float64
 12  liwc_ppron         38 non-null     float64
 13  liwc_i             38 non-null     float64
 14  liwc_we            38 non-null     float64
 15  liwc_you           38 non-null     float64
 16  liwc_shehe         38 non-

In [7]:
file['body_anon'] = file['body_anon'].astype(str)

In [16]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer


In [19]:
file['body_anon']

0                                                    Hey
1                                               Hey guys
2      Yeah I think we should: 71J871LWL3) Backup UAH...
3      Okay so as protection we will just clear all i...
4      Can M5P10WM4K7 the whole desktop with excel fi...
                             ...                        
155    I think it is a good idea if you guys can orga...
156                                             Hey guys
157    Tue =WG4V688FUG V1AGEWA8CT Wed =WG4V688FUG OAP...
158                                          Hey friends
159       I was thinking of not taking all of their data
Name: body_anon, Length: 160, dtype: object

In [182]:
def prep(dfx):
    dfx1 = [re.sub(r'[^\w\s]','', x) for x in dfx]
    dfx1 = ''.join(dfx1)
    return(dfx1)

df1 = file['body_anon'].apply(prep)

In [10]:
df1

NameError: name 'df1' is not defined

In [11]:
stops = set(stopwords.words('english'))
print(stops)

{'mightn', 'more', 'nor', 'ma', 'had', 'only', 'are', 'wouldn', 'shan', "that'll", 'all', 'because', 'they', 'whom', 'some', 'being', 'doesn', 'their', 'until', 're', 'yours', 'with', "couldn't", "aren't", "you're", 'on', 'having', 'itself', 'needn', 'again', 'too', "she's", 'between', 'down', 'hers', "hasn't", 'we', 'did', 'him', 'his', "you've", 'doing', "it's", 'in', 'y', 'will', 'under', 'yourself', 'what', 'to', 'them', 'you', 'that', 'up', 'into', 'wasn', 'myself', 'as', 'at', "mustn't", 'aren', 'who', 'your', 'don', 'he', 'themselves', 'herself', 'such', 'but', 'same', 't', 'haven', 'shouldn', 'is', 'weren', "wasn't", "isn't", 'own', 'hasn', "weren't", "mightn't", 'after', 'does', 'o', "won't", "you'll", 'should', 'if', "hadn't", 'our', 'about', 'here', "shouldn't", 'than', "wouldn't", 'so', "needn't", 'himself', 'am', 'further', 'll', 'which', 'be', 'was', 'or', 've', 'ain', 'before', 'below', 'out', 'it', 'those', 'while', 'any', 'above', 'me', 'most', 'then', 'few', 'has', 'n

In [24]:
def prepare_text(pretext):

    # tokenize mission.
    tokens = word_tokenize(pretext, language="english", preserve_line=True)

    # Parts of speech (POS) tag tokens.
    token_tag = pos_tag(tokens)
    
    # Only include some of the POS tags.
    include_tags = ['VBN', 'VBD', 'JJ', 'JJS', 'JJR', 'CD', 'NN', 'NNS', 'NNP', 'NNPS']
    filtered_tokens = (tok for tok, tag in token_tag if tag in include_tags)
    
    #stem words.
    stemmed_tokens = (lancaster.stem(tok) for tok in filtered_tokens)

    #lemm words.
    lemmed_words = (lemmatizer.lemmatize(tok) for tok in stemmed_tokens)

    lower_words = [str.lower(tok) for tok in lemmed_words]

    #x = cv.fit_transform(tokens).toarray()

    return(lower_words)

    #ps = PorterStemmer()
lancaster = LancasterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()
    
file['tokenized'] = file['body_anon'].apply(prepare_text)


In [25]:
file


,label,body_anon,liwc_WC,liwc_Analytic,liwc_Clout,liwc_Authentic,liwc_Tone,liwc_WPS,liwc_Sixltr,liwc_Dic,...,liwc_Colon,liwc_SemiC,liwc_QMark,liwc_Exclam,liwc_Dash,liwc_Quote,liwc_Apostro,liwc_Parenth,liwc_OtherP,tokenized
0,NORMAL,Hey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[hey]
1,NORMAL,Hey guys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[hey, guy]"
2,NORMAL,Yeah I think we should: 71J871LWL3) Backup UAH...,42.0,22.77,82.96,6.50,3.31,14.00,14.29,78.57,...,2.38,0.0,2.38,0.0,0.0,0.0,0.0,7.14,0.0,"[71j871lwl3, backup, uahqj3gkvd, password, she..."
3,NORMAL,Okay so as protection we will just clear all i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[okay, protect, inform, password, sheets., own..."
4,NORMAL,Can M5P10WM4K7 the whole desktop with excel fi...,38.0,12.16,96.70,50.35,97.58,9.50,7.89,81.58,...,0.00,0.0,2.63,0.0,0.0,0.0,0.0,0.00,0.0,"[whol, desktop, excel, files., log, sur, track..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,ANOMALY,I think it is a good idea if you guys can orga...,69.0,46.46,87.68,27.39,78.80,8.62,15.94,91.30,...,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,"[good, ide, meet, sess, contest., fac, team, o..."
156,ANOMALY,Hey guys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[hey, guy]"
157,ANOMALY,Tue =WG4V688FUG V1AGEWA8CT Wed =WG4V688FUG OAP...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[tue, =wg4v688fug, v1agewa8ct, wed, =wg4v688fu..."
158,ANOMALY,Hey friends,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[hey, friend]"
